In [1]:
import argparse
import os
import numpy as np
import torch
import torchvision
import torch.nn as nn
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2 
from PIL import Image
from unet import UNet

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_load = "/home/vinod/Documents/mitosis_detection/models/UNet/checkpoint_epoch100.pth"
model = UNet(in_chnls = 3, n_classes = 1)
model.load_state_dict(torch.load(model_load))
model.to(device=device)
model.eval()

UNet(
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (down_conv_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (down_conv_2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (down_conv_3): Sequential(
    (0): Conv2d(128, 256, kernel_size=

In [3]:
img_path = "/data1/vinod/mitosis/data/processed/val/input/A00_02.png"

In [15]:
img = Image.open(img_path)

test_transform = A.Compose([A.Resize(512,512),
                        A.Normalize(mean=(0,0,0),std=(1,1,1),max_pixel_value=255),
                        ToTensorV2()])

test_image = test_transform(image = np.asarray(img))
test_image = test_image["image"].unsqueeze(0)
test_image = test_image.to(device)

mask_path = img_path.replace("input","output")
mask = Image.open(mask_path)
mask = mask.resize((512, 512))
test_mask = np.copy(np.asarray(mask))
test_mask[test_mask==255] = 1

pred_mask = model(test_image)
pred_mask = pred_mask.squeeze(0).cpu().detach().numpy()
pred_mask = pred_mask.transpose(1,2,0)
pred_mask[pred_mask < 0]=0
pred_mask[pred_mask > 0]=1
pred_mask = np.squeeze(pred_mask)

In [19]:
def compute_iou(pred_mask, target_mask):
    # Flatten the masks
    pred_mask = pred_mask.flatten()
    target_mask = target_mask.flatten()

    # Calculate intersection and union
    intersection = np.logical_and(pred_mask, target_mask)
    union = np.logical_or(pred_mask, target_mask)

    # Compute IOU score
    iou = np.sum(intersection) / np.sum(union)
    return iou

In [18]:
iou_score = compute_iou(pred_mask, test_mask)
print("IOU score:", iou_score)

IOU score: 0.2786885245901639


In [28]:
def compute_pixel_accuracy(pred_mask, target_mask):
    # Flatten the masks
    pred_mask = pred_mask.flatten()
    target_mask = target_mask.flatten()

    # Calculate pixel accuracy
    pixel_accuracy = np.mean(pred_mask == target_mask)

    return pixel_accuracy



In [29]:
# Path to the folder containing predicted and target masks
folder_path = '/data1/vinod/mitosis/data/processed/val/input'

iou_scores = []
pixel_accuracies = []
for file_name in os.listdir(folder_path):
    if file_name.endswith('.png'):
        # Load predicted and target masks
        img_path = os.path.join(folder_path, file_name)
        
        img = Image.open(img_path)

        test_transform = A.Compose([A.Resize(512,512),
                                A.Normalize(mean=(0,0,0),std=(1,1,1),max_pixel_value=255),
                                ToTensorV2()])

        test_image = test_transform(image = np.asarray(img))
        test_image = test_image["image"].unsqueeze(0)
        test_image = test_image.to(device)

        mask_path = img_path.replace("input","output")
        mask = Image.open(mask_path)
        mask = mask.resize((512, 512))
        test_mask = np.copy(np.asarray(mask))
        test_mask[test_mask==255] = 1

        pred_mask = model(test_image)
        pred_mask = pred_mask.squeeze(0).cpu().detach().numpy()
        pred_mask = pred_mask.transpose(1,2,0)
        pred_mask[pred_mask < 0]=0
        pred_mask[pred_mask > 0]=1
        pred_mask = np.squeeze(pred_mask)

        # Compute IOU score for the current image
        iou_score = compute_iou(pred_mask, test_mask)
        iou_scores.append(iou_score)
        # Compute pixel accuracy for the current image
        pixel_accuracy = compute_pixel_accuracy(pred_mask, test_mask)
        pixel_accuracies.append(pixel_accuracy)

# Compute average IOU score
avg_iou_score = np.mean(iou_scores)
print("Average IOU score:", avg_iou_score)
# Compute average pixel accuracy
avg_pixel_accuracy = np.mean(pixel_accuracies)
print("Average pixel accuracy:", avg_pixel_accuracy)


Average IOU score: 0.23698373026450534
Average pixel accuracy: 0.9986399600380346


In [21]:
iou_scores

[]